### Determine highest predicted growth rates

In [78]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [79]:
model_read_dir = '../models/'

# Load in saved models for Greater Melbourne 
with open(f"{model_read_dir}gm_lr.pkl", 'rb') as file:
    gm_lr_model = pickle.load(file)

with open(f"{model_read_dir}gm_rf.pkl", 'rb') as file:
    gm_rf_model = pickle.load(file)
    
# Load in saved models for Rest of Vic
with open(f"{model_read_dir}rv_lr.pkl", 'rb') as file:
    rv_lr_model = pickle.load(file)

with open(f"{model_read_dir}rv_rf.pkl", 'rb') as file:
    rv_rf_model = pickle.load(file)

In [80]:
# Ensure the models have been correctly loaded in

print(f"Greater Melbourne Linear Regression Type: {type(gm_lr_model)}") 
print(f"Greater Melbourne Random Forest Type: {type(gm_rf_model)}") 
print(f"Rest of Vic Linear Regression Type: {type(rv_lr_model)}") 
print(f"Rest of Vic Random Forest Type: {type(rv_rf_model)}") 

Greater Melbourne Linear Regression Type: <class 'sklearn.linear_model._base.LinearRegression'>
Greater Melbourne Random Forest Type: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Rest of Vic Linear Regression Type: <class 'sklearn.linear_model._base.LinearRegression'>
Rest of Vic Random Forest Type: <class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [81]:
# Read in prediction data and their identifiers for later
forecast_read_dir = '../data/curated/forecast/'

gm_df = pd.read_csv(f"{forecast_read_dir}greater_melbourne_predict.csv")
gm_identifiers_df = pd.read_csv(f"{forecast_read_dir}gm_predict_identifiers.csv")

rv_df = pd.read_csv(f"{forecast_read_dir}rest_of_vic_predict.csv")
rv_identifiers_df = pd.read_csv(f"{forecast_read_dir}rv_predict_identifiers.csv")

In [82]:
# Apply necessary transformations on test dataset 

# Linear Regression Transformations for Greater Melbourne and Rest of Vic in the same order as 
# modelling_properties.ipynb

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
scaler = StandardScaler()

gm_lr_np = scaler.fit_transform(gm_df)
gm_lr_np = poly.fit_transform(gm_lr_np)

rv_lr_np = poly.fit_transform(rv_df)
rv_lr_np = scaler.fit_transform(rv_lr_np)

# Random Forest Transformation - only StandardScaler 
gm_rf_np = scaler.fit_transform(gm_df)
rv_rf_np = scaler.fit_transform(rv_df)

In [83]:
# Predict forecasted data

gm_lr_predictions = gm_lr_model.predict(gm_lr_np)
gm_rf_predictions = gm_rf_model.predict(gm_rf_np)

rv_lr_predictions = rv_lr_model.predict(rv_lr_np)
rv_rf_predictions = rv_rf_model.predict(rv_rf_np)

In [84]:
# Convert predictions to dataframe type
gm_lr_pred_df = pd.DataFrame(gm_lr_predictions, columns=['weekly_cost'])
gm_rf_pred_df = pd.DataFrame(gm_rf_predictions, columns=['weekly_cost'])

rv_lr_pred_df = pd.DataFrame(rv_lr_predictions, columns=['weekly_cost'])
rv_rf_pred_df = pd.DataFrame(rv_rf_predictions, columns=['weekly_cost'])

In [106]:
# Back-transform predictions, as they were initially log-transformed
gm_lr_orig_pred = np.exp(gm_lr_pred_df)
gm_rf_orig_pred = np.exp(gm_rf_pred_df)

rv_lr_orig_pred = np.exp(rv_lr_pred_df)
rv_rf_orig_pred = np.exp(rv_rf_pred_df)

/home/obli/.local/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: overflow encountered in expm1
  result = func(self.values, **kwargs)


In [86]:
print(type(gm_lr_orig_pred))

<class 'pandas.core.frame.DataFrame'>


In [108]:
# Join with identifiers for analysis
gm_lr_pred_final = pd.concat([gm_identifiers_df, gm_lr_orig_pred], axis=1)
gm_rf_pred_final = pd.concat([gm_identifiers_df, gm_rf_orig_pred], axis=1)

rv_lr_pred_final = pd.concat([rv_identifiers_df, rv_lr_orig_pred], axis=1)
rv_rf_pred_final = pd.concat([rv_identifiers_df, rv_rf_orig_pred], axis=1)

In [109]:
print("GM Linear Regression Prediction Head: ")
print(gm_lr_pred_final.head())
print("GM Random Forest Prediction Head: ")
print(gm_rf_pred_final.head())

print("RV Linear Regression Prediction Head: ")
print(rv_lr_pred_final.head())
print("RV Random Forest Prediction Head: ")
print(rv_rf_pred_final.head())

GM Linear Regression Prediction Head: 
        suburb  year  weekly_cost
0  albert park  2025   477.016849
1  albert park  2026   487.027498
2  albert park  2027   558.784444
3  albert park  2028   596.048514
4  albert park  2029   639.160690
GM Random Forest Prediction Head: 
        suburb  year  weekly_cost
0  albert park  2025   578.158865
1  albert park  2026   594.103149
2  albert park  2027   599.991177
3  albert park  2028   657.865106
4  albert park  2029   664.811661
RV Linear Regression Prediction Head: 
   suburb  year  weekly_cost
0  ararat  2025          inf
1  ararat  2026          inf
2  ararat  2027          inf
3  ararat  2028          inf
4  ararat  2029          inf
RV Random Forest Prediction Head: 
   suburb  year  weekly_cost
0  ararat  2025   346.963978
1  ararat  2026   358.998787
2  ararat  2027   379.761175
3  ararat  2028   464.450036
4  ararat  2029   535.201646


In [97]:
# Group by suburb, year and transform to have feature of predicted weekly cost per year for each
# suburb

gm_predicted_average_costs = gm_lr_pred_final.groupby(['suburb', 'year'])['weekly_cost'].mean().reset_index()
rv_predicted_average_costs = rv_lr_pred_final.groupby(['suburb', 'year'])['weekly_cost'].mean().reset_index()

# TransformingD
gm_pred_costs_transformed = gm_predicted_average_costs.pivot(index='suburb', columns='year', values='weekly_cost')
gm_pred_costs_transformed.columns = [f'{year}_weekly_cost' for year in gm_pred_costs_transformed.columns]  # Rename columns
gm_pred_costs_transformed.reset_index(inplace=True)  # Reset index to bring 'suburb' back as a column

rv_pred_costs_transformed = rv_predicted_average_costs.pivot(index='suburb', columns='year', values='weekly_cost')
rv_pred_costs_transformed.columns = [f'{year}_weekly_cost' for year in rv_pred_costs_transformed.columns]  # Rename columns
rv_pred_costs_transformed.reset_index(inplace=True)  # Reset index to bring 'suburb' back as a column

In [98]:
# Read in current data to compare 2024 prices
data_read_dir = '../data/curated/final_datasets/'

gm_real = pd.read_csv(f"{data_read_dir}greater_melbourne_train.csv")
gm_real_identifiers = pd.read_csv(f"{data_read_dir}gm_train_identifiers.csv")

rv_real = pd.read_csv(f"{data_read_dir}rest_of_vic_train.csv")
rv_real_identifiers = pd.read_csv(f"{data_read_dir}rv_train_identifiers.csv")

In [121]:
# Combine with identifiers and group by year, suburb, keep only 2024 data
gm_data = pd.concat([gm_real_identifiers, gm_real], axis=1)
rv_data = pd.concat([rv_real_identifiers, rv_real], axis=1)
###
gm_data = gm_data[gm_data['suburb'] == 'albert park']
gm_data[['weekly_cost', 'year']].head(50)
gm_data
###
# gm_current_average_costs = gm_data.groupby(['suburb', 'year'])['weekly_cost'].mean().reset_index()
# rv_current_average_costs = rv_data.groupby(['suburb', 'year'])['weekly_cost'].mean().reset_index()


# gm_current_average_costs = gm_current_average_costs[gm_current_average_costs['year'] == 2024]\
#     .rename(columns={'weekly_cost': '2024_weekly_cost'})\
#     .drop('year', axis=1)
# rv_current_average_costs = rv_current_average_costs[rv_current_average_costs['year'] == 2024]\
#     .rename(columns={'weekly_cost': '2024_weekly_cost'})\
#     .drop('year', axis=1)


,suburb,year,beds,baths,parking,dist_to_city,dist_to_education,dist_to_parks_and_gardens,dist_to_train_station,dist_to_healthcare,...,median_income,median_age,percent_aboriginal_torres_strait_islander,percent_au_citizen,percent_overseas_born,percent_rental_properties,population,percent_unemployed,housing_index,cpi_without_housing
0,albert park,2024,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,69755.0,42.46,0.313333,4.393333,35.14,43.72,14208.250639,4.573333,145.288421,127.830132
1,albert park,2024,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,69755.0,42.46,0.313333,4.393333,35.14,43.72,14208.250639,4.573333,145.288421,127.830132
2,albert park,2023,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,69358.0,42.12,0.303333,4.673333,34.76,43.58,13106.132992,4.543333,141.520263,125.573860
3,albert park,2021,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,68564.0,41.44,0.283333,5.233333,34.00,43.30,12696.762148,4.483333,133.983947,121.061316
4,albert park,2021,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,68564.0,41.44,0.283333,5.233333,34.00,43.30,12696.762148,4.483333,133.983947,121.061316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70409,albert park,2024,2.0,2.0,0.0,5.05500,0.49410,0.40409,3.99129,0.87577,...,69755.0,42.46,0.313333,4.393333,35.14,43.72,14208.250639,4.573333,145.288421,127.830132
71844,albert park,2024,2.0,2.0,0.0,4.50496,0.59895,0.33043,3.44125,0.37567,...,69755.0,42.46,0.313333,4.393333,35.14,43.72,14208.250639,4.573333,145.288421,127.830132
72542,albert park,2024,3.0,2.0,2.0,5.30167,0.65423,0.82612,4.23796,0.71617,...,69755.0,42.46,0.313333,4.393333,35.14,43.72,14208.250639,4.573333,145.288421,127.830132
72550,albert park,2024,1.0,1.0,0.0,4.39945,0.46451,0.16325,3.33574,0.37865,...,69755.0,42.46,0.313333,4.393333,35.14,43.72,14208.250639,4.573333,145.288421,127.830132


In [100]:
df = pd.read_csv(f"../data/curated/forecast/greater_melbourne_predict.csv")
df_identifiers = pd.read_csv(f"../data/curated/forecast/gm_predict_identifiers.csv")
ndf = pd.concat([df_identifiers, df], axis=1)

,suburb,year,beds,baths,parking,dist_to_city,dist_to_education,dist_to_parks_and_gardens,dist_to_train_station,dist_to_healthcare,...,median_income,median_age,percent_aboriginal_torres_strait_islander,percent_au_citizen,percent_overseas_born,percent_rental_properties,population,percent_unemployed,housing_index,cpi_without_housing
0,albert park,2025,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,70152.0,42.80,0.323333,4.113333,35.52,43.86,14420.682864,4.603333,149.056579,130.086404
1,albert park,2026,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,70549.0,43.14,0.333333,3.833333,35.90,44.00,14624.772379,4.633333,152.824737,132.342675
2,albert park,2027,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,70946.0,43.48,0.343333,3.553333,36.28,44.14,14820.286445,4.663333,156.592895,134.598947
3,albert park,2028,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,71343.0,43.82,0.353333,3.273333,36.66,44.28,15014.808184,4.693333,160.361053,136.855219
4,albert park,2029,2.0,1.0,1.0,4.42985,0.52632,0.19365,3.36613,0.44046,...,71740.0,44.16,0.363333,2.993333,37.04,44.42,15208.063939,4.723333,164.129211,139.111491


In [102]:
# TO DO: DEAL WITH WHY 2024 WEEKLY AVG COST IS HIGHER - HAVENT REMOVED EXPENSIVE top 5% properties 

# Merge data
gm_weekly_costs = pd.merge(gm_pred_costs_transformed, gm_current_average_costs, on='suburb', how='inner')
rv_weekly_costs = pd.merge(rv_pred_costs_transformed, rv_current_average_costs, on='suburb', how='inner')

YEARS = ["2025", "2026", "2027", "2028", "2029"]
for year in YEARS:
    gm_weekly_costs[f"{year}_percent_increase"] = gm_weekly_costs[f"{year}_weekly_cost"]/\
                                                  gm_weekly_costs['2024_weekly_cost']
    rv_weekly_costs[f"{year}_percent_increase"] = rv_weekly_costs[f"{year}_weekly_cost"]/\
                                                  rv_weekly_costs['2024_weekly_cost']

# gm_weekly_costs = gm_weekly_costs[gm_weekly_costs['2029_weekly_cost'] > gm_weekly_costs['2024_weekly_cost']]
#gm_weekly_costs.sort_values(by="2029_percent_increase", ascending=False).head(10)
gm_weekly_costs[gm_weekly_costs['suburb'] == "albert park"].head()


# gm_weekly_costs = gm_weekly_costs[gm_weekly_costs['2029_weekly_cost'] > gm_weekly_costs['2024_weekly_cost']]
#rv_weekly_costs.sort_values(by="2029_percent_increase", ascending=False).head(10)

,suburb,2025_weekly_cost,2026_weekly_cost,2027_weekly_cost,2028_weekly_cost,2029_weekly_cost,2024_weekly_cost,2025_percent_increase,2026_percent_increase,2027_percent_increase,2028_percent_increase,2029_percent_increase
5,albert park,609.0102,611.161957,716.514037,750.889393,791.078969,1095.208333,0.556068,0.558033,0.654226,0.685613,0.722309
